In [1]:
import sys
sys.path.append("../..")
from thesis.utils.utils import setup_ray, save, load, Experiment
path = "D:/Master/Masterarbeit/thesis"
setup_ray(path = path, unidirectional = True, seed=69)

2023-05-26 19:15:16,745	INFO worker.py:1528 -- Started a local Ray instance.


In [2]:
env_args = dict(
    fleetsize = 4,
    max_fleetsize = 20,    
    pseudo_routing = True,
    pseudo_dispatcher = False,
    pseudo_dispatcher_clever = False,
    #pseudo_dispatcher_distance = 0.3,
    routing_agent_death= False,
    death_on_target = False,
    transform_dispatching_partobs = True,
    direction_reward = 0,
    sim_config = dict(
        dispatch = True,
        routing_ma = True,
        dispatching_ma = True,
        reward_reached_target = 0,
        #reward_reached_target_by_time = True, 
        reward_wrong_target = 0,
        reward_removed_for_block = 0, 
        reward_target_distance = 0,
        reward_invalid= 0,
        reward_duration = -0.5,
        block_timeout = 120,
        station_separate = False,
        reward_accepted_in_station = 2,
        reward_declined_in_station = -1,
        #reward_part_completed = 5,
        reward_geo_operation=1,
        #reward_rework_operation=1,
        #reward_respot_operation=1,
        reward_reduce = -0.02,
        #reward_balance = -5,
        routing_interval = 2,
        dispatching_interval=360,
        io_quote = 0.9  ,
        availability = 0.9,
        mttr = 5,
        fixed_fleets = True,
    )
)

In [3]:
agv_model = dict(
    model = dict(
        custom_model = "gnn_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=16,
            with_action_mask=False,
            with_agvs=True,
            with_stations = False,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
            env_type = "matrix",
            n_convolutions = 2
        )
    )
)
dispatcher_model = dict(
    model = dict(
        custom_model = "lin_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=32,
            with_action_mask=True,
            with_agvs=True,
            with_stations = True,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
        
        )
    )
)

In [4]:
exp = Experiment("matrix_dispatching")
for seed in [43]:
    exp.experiment(
        path = path,
        env_args = env_args, 
        agv_model = agv_model,
        dispatcher_model=dispatcher_model,
        run_name="LAST", 
        env = "matrix",
        algo = "ppo",
        n_intervals =4,
        train_agv = False,
        backup_interval=20,
        batch_size=1000, #apex + gnn: 50
        seed = seed,
        algo_params = {"gamma":0.9},#,  "exploration_config":{"warmup_timesteps": 0,"epsilon_timesteps": 40000,"final_epsilon": 0.02,"initial_epsilon": 1.0,"type": "thesis.utils.masked_epsilon_greedy.MaskedEpsilonGreedy"}},# "exploration_config": {"type": "Curiosity", "sub_exploration": {"type": "StochasticSampling"}, "eta": 0.1}},
        lr = 1e-4,
        #load_agv="../../models/matrix_dispatching/10_new_matrix_4_20_2023-05-10_18-06-48/checkpoint_000060",
        two_fleets = True,
        
    )

2023-05-26 19:15:25,000	WARNING ppo.py:351 -- `train_batch_size` (1000) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=8 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 125.
2023-05-26 19:15:25,010	INFO algorithm.py:457 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_80238_2hdoio68)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
2023-05-26 19:15:45,291	INFO trainable.py:164 -- Trainable.setup took 20.293 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [8]:
save(exp.trainer, "dispatcher1", "../../models/fin_disp_1")
save(exp.trainer, "dispatcher2", "../../models/fin_disp_2")

In [6]:
#exp.keep_training(4)

In [7]:
#exp.keep_training(10)